3. Vấn đề 1: Tiến hành tải dữ liệu vào chương trình ứng dụng Python và giải quyết vấn đề
“Missing header in the csv file

In [3]:
import pandas as pd

column_names = ['Id','Name','Age','Weight','m0006','m0612','m1218','f0006','f0612','f1218']

df = pd.read_csv(
    "patient_heart_rate.csv",
    names=column_names,
    engine="python",
    on_bad_lines="warn"   # or "skip"
)

df.head()


/tmp/ipython-input-1506892307.py:5: ParserWarning: Skipping line 17: Expected 10 fields in line 17, saw 11

  df = pd.read_csv(


,Id,Name,Age,Weight,m0006,m0612,m1218,f0006,f0612,f1218
0,1.0,Mickéy Mousé,56.0,70kgs,72,69,71,-,-,-
1,2.0,Donald Duck,34.0,154.89lbs,-,-,-,85,84,76
2,3.0,Mini Mouse,16.0,NaN,-,-,-,65,69,72
3,4.0,Scrooge McDuck,NaN,78kgs,78,79,72,-,-,-
4,5.0,Pink Panther,54.0,198.658lbs,-,-,-,69,NaN,75


4. Vấn đề 2: Xử lý vấn đề một cột lưu hỗn hợp nhiều dữ liệu, ở đây là cột “Name” chứa bao
gồm “Firstname” và “Lastname”, giải pháp là ta sẽ tách ra làm 2 cột

In [4]:
df[['Firstname','Lastname']] = df['Name'].astype(str).str.split(expand=True, n=1)
df.drop(columns=['Name'], inplace=True)

print(df.head())


    Id   Age      Weight m0006 m0612 m1218 f0006 f0612 f1218 Firstname  \
0  1.0  56.0       70kgs    72    69    71     -     -     -    Mickéy   
1  2.0  34.0   154.89lbs     -     -     -    85    84    76    Donald   
2  3.0  16.0         NaN     -     -     -    65    69    72      Mini   
3  4.0   NaN       78kgs    78    79    72     -     -     -   Scrooge   
4  5.0  54.0  198.658lbs     -     -     -    69   NaN    75      Pink   

  Lastname  
0    Mousé  
1     Duck  
2    Mouse  
3   McDuck  
4  Panther  


5. Vấn đề 3: Cột Weight có vấn đề về không thống nhất các đơn vị đo lường trong dữ liệu.
Ta sẽ chuyển các đơn vị về thành đơn vị chuẩn “kg”

In [5]:
def weight_to_kgs(x):
    if pd.isna(x): return np.nan
    s = str(x).strip().lower()
    if s in ["", "-"]: return np.nan

    m = re.search(r"(\d+(\.\d+)?)", s)
    if not m: return np.nan
    val = float(m.group(1))

    if "lb" in s:  # lbs -> kgs (lab style: /2.2 then int)
        val = val / 2.2

    return f"{int(val)}kgs"

df["Weight"] = df["Weight"].apply(weight_to_kgs)

print(df.head())


    Id   Age Weight m0006 m0612 m1218 f0006 f0612 f1218 Firstname Lastname
0  1.0  56.0  70kgs    72    69    71     -     -     -    Mickéy    Mousé
1  2.0  34.0  70kgs     -     -     -    85    84    76    Donald     Duck
2  3.0  16.0    NaN     -     -     -    65    69    72      Mini    Mouse
3  4.0   NaN  78kgs    78    79    72     -     -     -   Scrooge   McDuck
4  5.0  54.0  90kgs     -     -     -    69   NaN    75      Pink  Panther


6. Vấn đề 4: Vấn đề về xuất hiện dòng dữ liệu rỗng (không có giá trị: NaN). Giải pháp có
thể đưa ra là xóa bỏ


In [7]:
df.dropna(how="all", inplace=True)

print(df.head())

    Id   Age Weight m0006 m0612 m1218 f0006 f0612 f1218 Firstname Lastname
0  1.0  56.0  70kgs    72    69    71     -     -     -    Mickéy    Mousé
1  2.0  34.0  70kgs     -     -     -    85    84    76    Donald     Duck
5  6.0  52.0  85kgs     -     -     -    68    75    72      Huey   McDuck
6  7.0  19.0  56kgs     -     -     -    71    78    75     Dewey   McDuck
7  8.0  32.0  78kgs    78    76    75     -     -     -    Scööpy      Doo


7. Vấn đề 5: Có nhiều dòng dữ liệu bị trùng lắp thông tin hoàn toàn[fullname, lastname,
age, weight,....], giải pháp đưa ra là chỉ giữ lại một dòng dữ liệu, tuy nhiên giải pháp phải
dựa trên nghiệp vụ của tập dữ liệu và quan sát của người xử lý.


In [8]:
df = df.drop_duplicates(subset=["Firstname", "Lastname", "Age", "Weight"])

print(df.head())

    Id   Age Weight m0006 m0612 m1218 f0006 f0612 f1218 Firstname Lastname
0  1.0  56.0  70kgs    72    69    71     -     -     -    Mickéy    Mousé
1  2.0  34.0  70kgs     -     -     -    85    84    76    Donald     Duck
5  6.0  52.0  85kgs     -     -     -    68    75    72      Huey   McDuck
6  7.0  19.0  56kgs     -     -     -    71    78    75     Dewey   McDuck
7  8.0  32.0  78kgs    78    76    75     -     -     -    Scööpy      Doo


8. Vấn đề 6: Xuất hiện dữ liệu bị ảnh hưởng bởi lỗi non-ASCII, không định dạng ASCII.
Giải pháp: Tùy vào nghiệp vụ ta có thể: xóa dữ liệu tại đó, thay thế bằng dữ liệu khác
hoặc thay bằng việc đánh dấu bằng một kí tự khác (ví dụ: ‘warning’)

In [9]:
df.Firstname.replace({r'[^\x00-\x7F]+':''}, regex=True, inplace=True)
df.Lastname.replace({r'[^\x00-\x7F]+':''}, regex=True, inplace=True)

print(df.head)

<bound method NDFrame.head of       Id   Age Weight m0006 m0612 m1218 f0006 f0612 f1218 Firstname Lastname
0    1.0  56.0  70kgs    72    69    71     -     -     -     Micky     Mous
1    2.0  34.0  70kgs     -     -     -    85    84    76    Donald     Duck
5    6.0  52.0  85kgs     -     -     -    68    75    72      Huey   McDuck
6    7.0  19.0  56kgs     -     -     -    71    78    75     Dewey   McDuck
7    8.0  32.0  78kgs    78    76    75     -     -     -      Scpy      Doo
11  10.0  12.0  45kgs     -     -     -    92    95    87     Louie   McDuck>


9. Vấn đề 7

In [10]:
# Convert "-" and "" to NaN for numeric handling
def norm_missing(x):
    if pd.isna(x): return np.nan
    s = str(x).strip()
    return np.nan if s in ["", "-"] else s

# Age to numeric
df["Age"] = pd.to_numeric(df["Age"].apply(norm_missing), errors="coerce")

# Make a numeric weight column for logic (extract number from "70kgs")
df["Weight_kg"] = pd.to_numeric(df["Weight"].astype(str).str.extract(r"(\d+)")[0], errors="coerce")

# Drop rows missing BOTH Age and Weight
mask_drop = df["Age"].isna() & df["Weight_kg"].isna()
df = df[~mask_drop].reset_index(drop=True)

# Fill missing Age with mean (as lab suggests)
df["Age"] = df["Age"].fillna(df["Age"].mean())

# Heart rate columns -> numeric, "-" -> NaN
hr_cols = ['m0006','m0612','m1218','f0006','f0612','f1218']
for c in hr_cols:
    df[c] = pd.to_numeric(df[c].apply(norm_missing), errors="coerce")

print(df.head())
print("Missing by column:\n", df.isna().sum())


    Id   Age Weight  m0006  m0612  m1218  f0006  f0612  f1218 Firstname  \
0  1.0  56.0  70kgs   72.0   69.0   71.0    NaN    NaN    NaN     Micky   
1  2.0  34.0  70kgs    NaN    NaN    NaN   85.0   84.0   76.0    Donald   
2  6.0  52.0  85kgs    NaN    NaN    NaN   68.0   75.0   72.0      Huey   
3  7.0  19.0  56kgs    NaN    NaN    NaN   71.0   78.0   75.0     Dewey   
4  8.0  32.0  78kgs   78.0   76.0   75.0    NaN    NaN    NaN      Scpy   

  Lastname  Weight_kg  
0     Mous         70  
1     Duck         70  
2   McDuck         85  
3   McDuck         56  
4      Doo         78  
Missing by column:
 Id           0
Age          0
Weight       0
m0006        4
m0612        4
m1218        4
f0006        2
f0612        2
f1218        2
Firstname    0
Lastname     0
Weight_kg    0
dtype: int64


10. Vấn đề 8

In [30]:
import re
import pandas as pd
import numpy as np

# 1) Auto-detect heart-rate columns like m0006, f1218, m0001...
hr_cols = [c for c in df.columns if re.match(r'^[mf]\d{4}$', str(c))]

print("Detected heart rate columns:", hr_cols)

# 2) Safety: make sure required id columns exist
id_vars = [c for c in ['Id','Age','Weight','Firstname','Lastname'] if c in df.columns]
print("Detected id vars:", id_vars)

# 3) Melt
df8 = pd.melt(
    df,
    id_vars=id_vars,
    value_vars=hr_cols,
    var_name='sex_and_time',
    value_name='PulseRateValue'
).reset_index(drop=True)

# 4) Extract Sex + time from column name
tmp = df8['sex_and_time'].str.extract(r'([mf])(\d{2})(\d{2})', expand=True)
tmp.columns = ['Sex', 'hours_lower', 'hours_upper']
df8['Sex'] = tmp['Sex']
df8['Time'] = tmp['hours_lower'] + "-" + tmp['hours_upper']

# 5) Clean PulseRate
df8['PulseRate'] = pd.to_numeric(
    df8['PulseRateValue'].replace(['-', ''], np.nan),
    errors='coerce'
)

df8 = df8.drop(columns=['sex_and_time','PulseRateValue'])

df8 = df8.sort_values(id_vars + ['Sex','Time']).reset_index(drop=True)
df8.head(12)


Detected heart rate columns: []
Detected id vars: ['Id', 'Age', 'Weight', 'Firstname', 'Lastname']


,Id,Age,Weight,Firstname,Lastname,Sex,Time,PulseRate


In [29]:
MED_DEFAULT = 70

df = df.sort_values(["Id","Sex","Time"]).reset_index(drop=True)

df["PulseRate"] = df.groupby(["Id","Sex"])["PulseRate"].ffill()
df["PulseRate"] = df.groupby(["Id","Sex"])["PulseRate"].bfill()
df["PulseRate"] = df["PulseRate"].fillna(df.groupby("Sex")["PulseRate"].transform("mean"))
df["PulseRate"] = df["PulseRate"].fillna(df["PulseRate"].mean())
df["PulseRate"] = df["PulseRate"].fillna(MED_DEFAULT)

df.isna().sum()


,0
Id,0
Firstname,0
Lastname,0
Age,0
Weight,0
sex_and_time,0
PulseRate,0
Sex,6
hourse_lower,6
hours_upper,6


In [28]:
import re
import pandas as pd
import numpy as np

# 0) If df already long-format, stop
if set(["Sex","Time","PulseRate"]).issubset(df.columns):
    print("df is already in long format (Sex/Time/PulseRate exist). Skip Problem 8.")
else:
    # 1) Auto-detect heart rate columns: m0006, f1218, m0001, ...
    hr_cols = [c for c in df.columns if re.match(r'^[mf]\d{4}$', str(c))]
    if len(hr_cols) == 0:
        raise ValueError("No heart-rate columns found. Check Problem 1 column_names / df.columns.")

    # 2) Pick id columns that actually exist
    preferred_ids = ['Id','Age','Weight','Firstname','Lastname']
    id_vars = [c for c in preferred_ids if c in df.columns]
    if len(id_vars) == 0:
        raise ValueError("No id_vars found. Your df must have at least Id (and usually Age/Weight/Firstname/Lastname).")

    # 3) Melt wide -> long (avoid name clash)
    df8 = pd.melt(
        df,
        id_vars=id_vars,
        value_vars=hr_cols,
        var_name="sex_and_time",
        value_name="PulseRateValue"
    ).reset_index(drop=True)

    # 4) Extract Sex + Time from column name (m0006 -> Sex=m, Time=00-06)
    tmp = df8["sex_and_time"].str.extract(r'^([mf])(\d{2})(\d{2})$')
    df8["Sex"] = tmp[0]
    df8["Time"] = tmp[1] + "-" + tmp[2]

    # 5) Clean PulseRate to numeric
    df8["PulseRate"] = pd.to_numeric(
        df8["PulseRateValue"].replace(["-", ""], np.nan),
        errors="coerce"
    )

    # 6) Final tidy
    df8 = df8.drop(columns=["sex_and_time", "PulseRateValue"])
    df8 = df8.sort_values(id_vars + ["Sex","Time"]).reset_index(drop=True)

    print("Detected heart rate columns:", hr_cols)
    display(df8.head(12))


df is already in long format (Sex/Time/PulseRate exist). Skip Problem 8.


11. vấn đề 9

In [26]:
df_long["PulseRate"] = df_long.groupby(["Id","Sex"])["PulseRate"].ffill()
print(df_long.isna().sum()[["PulseRate"]])


PulseRate    18
dtype: int64


12.

In [27]:
df_long["PulseRate"] = df_long.groupby(["Id","Sex"])["PulseRate"].bfill()
print(df_long.isna().sum()[["PulseRate"]])


PulseRate    18
dtype: int64


In [31]:
MED_DEFAULT = 70

df_long["PulseRate"] = df_long["PulseRate"].fillna(df_long.groupby("Sex")["PulseRate"].transform("mean"))
df_long["PulseRate"] = df_long["PulseRate"].fillna(df_long["PulseRate"].mean())
df_long["PulseRate"] = df_long["PulseRate"].fillna(MED_DEFAULT)

print(df_long.isna().sum()[["PulseRate"]])
df_long.head(12)


PulseRate    0
dtype: int64


,Id,Firstname,Lastname,Age,Weight,Weight_kg,PulseRate,Sex,Time
0,1.0,Micky,Mous,56.0,70kgs,70,79.833333,f,6
1,1.0,Micky,Mous,56.0,70kgs,70,79.833333,f,612
2,1.0,Micky,Mous,56.0,70kgs,70,79.833333,f,1218
3,1.0,Micky,Mous,56.0,70kgs,70,72.000000,m,6
4,1.0,Micky,Mous,56.0,70kgs,70,69.000000,m,612
5,1.0,Micky,Mous,56.0,70kgs,70,71.000000,m,1218
6,2.0,Donald,Duck,34.0,70kgs,70,85.000000,f,6
7,2.0,Donald,Duck,34.0,70kgs,70,84.000000,f,612
8,2.0,Donald,Duck,34.0,70kgs,70,76.000000,f,1218
9,2.0,Donald,Duck,34.0,70kgs,70,73.500000,m,6


In [33]:
# Problem 12: Reduce columns + reindex + save
OUT = "patient_heart_rate_clean.csv"

final_cols = [c for c in ["Id","Firstname","Lastname","Age","Weight","Sex","Time","PulseRate"] if c in df.columns]
df_clean = df[final_cols].copy().reset_index(drop=True)

df_clean.to_csv(OUT, index=False)
print("Saved:", OUT)
df_clean.head(10)


Saved: patient_heart_rate_clean.csv


,Id,Firstname,Lastname,Age,Weight,Sex,Time,PulseRate
0,1.0,Micky,Mous,56.0,70kgs,f,00:06,79.833333
1,1.0,Micky,Mous,56.0,70kgs,f,06:12,79.833333
2,1.0,Micky,Mous,56.0,70kgs,f,12:18,79.833333
3,1.0,Micky,Mous,56.0,70kgs,m,00:06,72.000000
4,1.0,Micky,Mous,56.0,70kgs,m,06:12,69.000000
5,1.0,Micky,Mous,56.0,70kgs,m,12:18,71.000000
6,1.0,Micky,Mous,56.0,70kgs,NaN,NaN,76.666667
7,2.0,Donald,Duck,34.0,70kgs,f,00:06,85.000000
8,2.0,Donald,Duck,34.0,70kgs,f,06:12,84.000000
9,2.0,Donald,Duck,34.0,70kgs,f,12:18,76.000000
